In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import h5py
import os

def load_data_from_h5(h5_path, dataset_key='df/block0_values'):
    """
    Loads data from an HDF5 (.h5) file.
    If dataset_key is None, loads the first dataset.
    """
    with h5py.File(h5_path, 'r') as f:
        # Split the dataset_key to handle nested groups
        keys = dataset_key.split('/')
        group = f
        for key in keys[:-1]:
            group = group.get(key)
       
        # Now, the last part of keys is the dataset name
        dataset = group.get(keys[-1])

        if isinstance(dataset, h5py.Dataset):
            data = dataset[()]  # Safely load the dataset
            print(f"Loaded data shape: {data.shape}")
        else:
            raise TypeError(f"Dataset {dataset_key} is not of the expected type (h5py.Dataset), found: {type(dataset)}")
   
    return data

def fill_missing(data, method='zero'):
    data = data.astype(float)
    if method == 'zero':
        return np.nan_to_num(data)
    elif method == 'mean':
        nan_mask = np.isnan(data)
        col_mean = np.nanmean(data, axis=0)
        data[nan_mask] = np.take(col_mean, np.where(nan_mask)[1])
        return data
    else:
        raise ValueError("Unsupported fill method")

def normalize_data(data, scaler=None):
    original_shape = data.shape
    data_2d = data.reshape(-1, data.shape[-1])
    if scaler is None:
        scaler = MinMaxScaler()
        data_2d = scaler.fit_transform(data_2d)
    else:
        data_2d = scaler.transform(data_2d)
    return data_2d.reshape(original_shape), scaler

def create_sliding_windows(data, window_size, horizon):
    X, Y = [], []
    for i in range(data.shape[0] - window_size - horizon + 1):
        x_i = data[i:i+window_size]
        y_i = data[i+window_size:i+window_size+horizon]
        X.append(x_i)
        Y.append(y_i)
    X = np.stack(X)[:, :, :, np.newaxis]
    Y = np.stack(Y)[:, :, :, np.newaxis]
    return X, Y

def main():
    # Define the h5 file path
    h5_file = os.path.expanduser('~/Desktop/TrafficPrediction/Enhanced/data/METR.h5')

    # Check if the file exists
    if not os.path.exists(h5_file):
        raise FileNotFoundError(f"The file {h5_file} was not found.")

    # Load data with the correct dataset key
    data = load_data_from_h5(h5_file, dataset_key='df/block0_values')  # Correct dataset_key

    # Handle missing values by filling with the mean
    data = fill_missing(data, method='mean')

    # Normalize data to [0, 1] range
    data, scaler = normalize_data(data)

    # Create sliding windows for training
    window_size = 12  # Past 1 hour of data (assuming 5-minute intervals, 12 * 5 = 60 minutes)
    horizon = 12      # Predict the next 1 hour (12 * 5 = 60 minutes)
    X, Y = create_sliding_windows(data, window_size, horizon)

    print(f"X shape: {X.shape} (samples, time_steps, nodes, features)")
    print(f"Y shape: {Y.shape} (prediction targets)")

    # Save the preprocessed data as NumPy files for use in the Dataset class
    np.save('X.npy', X)
    np.save('Y.npy', Y)
    print("Preprocessed data saved as X.npy and Y.npy")

if __name__ == '__main__':
    main()

In [ ]:
import torch
from torch.utils.data import Dataset

class TrafficDataset(Dataset):
    def __init__(self, x_path, y_path):
        # Load the preprocessed data
        self.X = np.load(x_path)  # shape: (samples, time_steps, nodes, 1)
        self.Y = np.load(y_path)  # shape: (samples, horizon, nodes, 1)

        # Convert to torch tensors and permute to [samples, features, time, nodes]
        self.X = torch.from_numpy(self.X).float().permute(0, 3, 1, 2)  # → [N, 1, 12, 500]
        self.Y = torch.from_numpy(self.Y).float().permute(0, 3, 1, 2)  # → [N, 1, 3, 500]

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]


In [ ]:
from torch.utils.data import DataLoader, random_split

# Load dataset
dataset = TrafficDataset('X.npy', 'Y.npy')

# Split into train, val, test
train_size = int(0.7 * len(dataset))
val_size = int(0.15 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_set, val_set, test_set = random_split(dataset, [train_size, val_size, test_size])

# Create loaders
train_loader = DataLoader(train_set, batch_size=64, shuffle=True)
val_loader = DataLoader(val_set, batch_size=64, shuffle=False)
test_loader = DataLoader(test_set, batch_size=64, shuffle=False)


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

class GCNLayer(nn.Module):
    def __init__(self, adj_matrix, in_channels, out_channels):
        super(GCNLayer, self).__init__()
        self.adj_matrix = adj_matrix  # Adjacency matrix with shape [N, N]
        self.in_channels = in_channels
        self.out_channels = out_channels
        
        # Learnable transformation matrix for node features
        self.theta = nn.Parameter(torch.randn(in_channels, out_channels))  # [F_in, F_out]

    def forward(self, x):
        """
        x shape: [B, T, N, F_in]
        adj_matrix shape: [N, N] (with N nodes)
        """
        B, T, N, F_in = x.shape
        
        # Expand adjacency matrix to match the batch and time dimensions
        adj_matrix_expanded = self.adj_matrix.unsqueeze(0).unsqueeze(0).expand(B, T, N, N)  # [B, T, N, N]

        # Propagate node features based on the adjacency matrix
        # Apply adjacency matrix to propagate information between nodes for each time step and batch
        x = torch.einsum('btnf,btnn->btnf', x, adj_matrix_expanded)  # [B, T, N, F_in] x [B, T, N, N] -> [B, T, N, F_in]
        
        # Apply the learned transformation matrix (theta) to the node features
        x = torch.einsum('btnf,fo->btno', x, self.theta)  # [B, T, N, F_in] x [F_in, F_out] -> [B, T, N, F_out]
        
        return x



In [ ]:
class TransformerLayer(nn.Module):
    def __init__(self, input_dim, nhead, num_layers, dim_feedforward=512):
        super(TransformerLayer, self).__init__()
        self.transformer = nn.Transformer(
            d_model=input_dim,
            nhead=nhead,
            num_encoder_layers=num_layers,
            dim_feedforward=dim_feedforward,
            batch_first=True
        )

    def forward(self, x):
        # x shape: [B, T, N, F_in]
        B, T, N, F_in = x.shape

        # Reshape x to fit transformer input shape [B * N, T, F_in]
        x = x.view(B * N, T, F_in)

        # Pass through the transformer
        x = self.transformer(x, x)  # Self-attention on both source and target

        # Reshape back to [B, T, N, F_out]
        x = x.view(B, T, N, -1)
        return x

In [ ]:
class SpatioTemporalModel(nn.Module):
    def __init__(self, adj_matrix, gcn_in_channels, gcn_out_channels, transformer_input_dim, nhead, num_layers, num_classes):
        super(SpatioTemporalModel, self).__init__()
        
        # Initialize the GCN layer with the correct order of arguments
        self.gcn = GCNLayer(adj_matrix, gcn_in_channels, gcn_out_channels)  # Corrected argument order
        
        # Initialize Transformer layer
        self.transformer = TransformerLayer(transformer_input_dim, nhead, num_layers)
        
        # Fully connected layer for output prediction
        self.fc = nn.Linear(transformer_input_dim, num_classes)
        
    def forward(self, x):
        x = self.gcn(x)  # Apply GCN layer
        x = self.transformer(x)  # Apply Transformer layer
        x = x[:, -1, :, :]  # Use the last time step for prediction
        x = self.fc(x)  # Final prediction layer
        return x


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Training function
def train_model(model, train_loader, val_loader, epochs=100, lr=0.001):
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for X_batch, Y_batch in train_loader:
            optimizer.zero_grad()
            Y_pred = model(X_batch)
            loss = criterion(Y_pred, Y_batch)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for X_batch, Y_batch in val_loader:
                Y_pred = model(X_batch)
                loss = criterion(Y_pred, Y_batch)
                val_loss += loss.item()

        print(f"Epoch {epoch+1}/{epochs}, Train Loss: {running_loss/len(train_loader):.4f}, Validation Loss: {val_loss/len(val_loader):.4f}")

def calculate_metrics(model, data_loader):
    model.eval()
    y_true = []
    y_pred = []
   
    with torch.no_grad():
        for X_batch, Y_batch in data_loader:
            Y_pred = model(X_batch)
            y_true.append(Y_batch.cpu().numpy())
            y_pred.append(Y_pred.cpu().numpy())
   
    y_true = np.concatenate(y_true).flatten()
    y_pred = np.concatenate(y_pred).flatten()
   
    # Calculate MAE (Mean Absolute Error)
    mae = mean_absolute_error(y_true, y_pred)
   
    # Calculate RMSE (Root Mean Squared Error)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
   
    # Calculate MAPE (Mean Absolute Percentage Error)
    # Handle division by zero by masking non-zero true values
    nonzero_mask = y_true != 0
    if np.any(nonzero_mask):
        mape = np.mean(np.abs((y_true[nonzero_mask] - y_pred[nonzero_mask]) / y_true[nonzero_mask])) * 10
    else:
        mape = float('nan')  # MAPE undefined if all true values are zero
   
    print("\n=== Evaluation Metrics ===")
    print(f"MAE: {mae:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"MAPE: {mape:.4f}%")  # Display as percentage
   
    return mae, rmse, mape

def calculate_accuracy(model, data_loader, tolerance=0.05):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for X_batch, Y_batch in data_loader:
            Y_pred = model(X_batch)
            Y_pred_flat = Y_pred.view(-1)
            Y_batch_flat = Y_batch.view(-1)
            correct += ((torch.abs(Y_pred_flat - Y_batch_flat) < tolerance).sum().item())
            total += Y_batch_flat.numel()
    accuracy = correct / total
    print(f"Accuracy (±{tolerance} tolerance): {accuracy:.4f}")

# Dummy setup (replace with real data and model)
N = 500  # Nodes
B, T, F_in = 32, 10, 16
adj_matrix = torch.rand(N, N)

X_train = torch.randn(B, T, N, F_in)
Y_train = torch.randn(B, N, 1)

train_data = TensorDataset(X_train, Y_train)
train_loader = DataLoader(train_data, batch_size=B, shuffle=True)

# Replace this with your actual SpatioTemporalModel definition
model = SpatioTemporalModel(adj_matrix=adj_matrix, gcn_in_channels=F_in, gcn_out_channels=64,
                            transformer_input_dim=64, nhead=4, num_layers=2, num_classes=1)

# Train and evaluate
train_model(model, train_loader, train_loader, epochs=100)
calculate_metrics(model, train_loader)
calculate_accuracy(model, train_loader)
